# SentinelHub Playground

example taken from https://github.com/sentinel-hub/sentinelhub-py/blob/master/examples/ogc_request.ipynb

## Setup

In [ ]:
INSTANCE_ID = '0e7e4724-e954-4e7c-87ba-708bc8926c79'  # create account and sample layer via https://sentinel-hub.com/

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import datetime
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sentinelhub import WmsRequest, WcsRequest, FisRequest, MimeType, CRS, BBox, CustomUrlParam
from sentinelhub.time_utils import iso_to_datetime
list(CustomUrlParam)

In [ ]:
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))
    
    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

In [ ]:
betsiboka_coords_wgs84 = [46.16, -16.15, 46.51, -15.58]

In [ ]:
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)

### WMS (Python)

In [ ]:
wms_true_color_request = WmsRequest(layer='TRUE_COLOR',
                                    bbox=betsiboka_bbox,
                                    time=('2017-12-01', '2017-12-31'), # or <explicit date> or 'latest'
                                    width=512, height=856, 
                                    maxcc=0.3, # optional
                                    instance_id=INSTANCE_ID)

In [ ]:
wms_true_color_img = wms_true_color_request.get_data()

In [ ]:
print('There are %d Sentinel-2 images available for December 2017 with cloud coverage less ' \
      'than %1.0f%%.' % (len(wms_true_color_img), wms_true_color_request.maxcc * 100.0))

In [ ]:
plot_image(wms_true_color_img[-1])

In [ ]:
print('These %d images were taken on the following dates:' % len(wms_true_color_img))
for index, date in enumerate(wms_true_color_request.get_dates()):
    print(' - image %d was taken on %s' % (index, date))

In [ ]:
wms_bands_request = WmsRequest(layer='BANDS',
                               bbox=betsiboka_bbox, 
                               time='2017-12-15',
                               width=512, height=856,
                               image_format=MimeType.TIFF_d32f,
                               instance_id=INSTANCE_ID)
wms_bands_img = wms_bands_request.get_data()

In [ ]:
wms_bands_img[-1][:, :, 12].shape

In [ ]:
plot_image(wms_bands_img[-1][:, :, 12])

In [ ]:
plot_image(wms_bands_img[-1][:, :, [3, 2, 1]], 2.5)

In [ ]:
my_evalscript = '''
return [B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B10,B11,B12];
'''

evalscript_wms_request = WmsRequest(layer='TRUE_COLOR', # not important which one, just must exist
                                    bbox=betsiboka_bbox, 
                                    time='2017-12-20', 
                                    width=512,
                                    image_format=MimeType.TIFF_d32f,
                                    instance_id=INSTANCE_ID,
                                    custom_url_params={CustomUrlParam.EVALSCRIPT: my_evalscript})

evalscript_wms_data = evalscript_wms_request.get_data()
plot_image(evalscript_wms_data[-1][:, :, 12])

### WCS (Python)

In [ ]:
wcs_true_color_request = WcsRequest(layer='TRUE_COLOR',
                                    bbox=betsiboka_bbox, 
                                    time='2017-12-15', 
                                    resx='60m', resy='60m',
                                    instance_id=INSTANCE_ID)

In [ ]:
wcs_true_color_img = wcs_true_color_request.get_data()

In [ ]:
print('Single element in the list is of type = {} and has shape {}'.format(type(wcs_true_color_img[-1]),
                                                                           wcs_true_color_img[-1].shape))

In [ ]:
plot_image(wcs_true_color_img[-1])

### WMS (Http)

In [ ]:
wfs_params = {'SERVICE':'wfs','REQUEST':'GetFeature','TYPENAMES':'S2.TILE','BBOX':'-16.15,46.16,-15.58,46.51','OUTPUTFORMAT':'application/json','SRSNAME':'EPSG:4326','TIME':'2017-12-01/2017-12-31','MAXCC':'30.0','MAXFEATURES':'100','FEATURE_OFFSET':'0'}
wfs = requests.get("https://services.sentinel-hub.com/ogc/wfs/0e7e4724-e954-4e7c-87ba-708bc8926c79?", params=wfs_params)
wfs.url


In [ ]:
wfs.json()

In [ ]:
wms_params = {'SERVICE':'wms','MAXCC':'30.0','BBOX':'-16.15,46.16,-15.58,46.51','FORMAT':'image/png','CRS':'EPSG:4326','TIME':'2017-12-15T07:12:03/2017-12-15T07:12:03','WIDTH':'512','HEIGHT':'856','LAYERS':'TRUE_COLOR','REQUEST':'GetMap','VERSION':'1.3.0'}
wms = requests.get("https://services.sentinel-hub.com/ogc/wms/0e7e4724-e954-4e7c-87ba-708bc8926c79?", params=wms_params)
wms.url


In [ ]:
wms.raw

### FIS (Python)

In [ ]:
def fis_data_to_dataframe(fis_data):
    """ Creates a DataFrame from list of FIS responses
    """
    COLUMNS = ['channel', 'date', 'min', 'max', 'mean', 'stDev']
    data = []
    
    for fis_response in fis_data:
        for channel, channel_stats in fis_response.items():
            for stat in channel_stats:
                row = [int(channel[1:]), iso_to_datetime(stat['date'])]
                
                for column in COLUMNS[2:]:
                    row.append(stat['basicStats'][column])
                
                data.append(row)
    
    return pd.DataFrame(data, columns=COLUMNS).sort_values(['channel', 'date'])

In [ ]:
fis_request = FisRequest(layer='NDVI_INDEX',
                         geometry_list=[betsiboka_bbox],
                         time=('2017-12-01', '2017-12-31'),
                         resolution='60m',
                         instance_id=INSTANCE_ID)

In [ ]:
fis_data = fis_request.get_data()
fis_data[0]['C0'][0]

In [ ]:
df = fis_data_to_dataframe(fis_data)
df

In [ ]:
fis_histogram_request = FisRequest(layer='NDVI_INDEX',
                         geometry_list=[betsiboka_bbox],
                         time=('2017-12-01', '2017-12-31'),
                         resolution='60m',
                         bins=10,
                         instance_id=INSTANCE_ID)

In [ ]:
fis_histogram_data = fis_histogram_request.get_data()
fis_histogram_data[0]['C0'][0]

### Temporal

In [ ]:
parcel_coords_wgs84 = [16.5575304072,48.2110832663,16.5585174602,48.211975525]
parcel_bbox = BBox(bbox=parcel_coords_wgs84, crs=CRS.WGS84)

In [ ]:
wms_true_color_parcel_request = WmsRequest(layer='TRUE_COLOR',
                                    bbox=parcel_bbox,
                                    time='latest',
                                    width=512, height=856, 
                                    maxcc=0.3, # optional
                                    instance_id=INSTANCE_ID)

In [ ]:
wms_true_color_parcel_img = wms_true_color_parcel_request.get_data()

In [ ]:
plot_image(wms_true_color_parcel_img[-1])

In [ ]:
fis_parcel_request = FisRequest(layer='NDVI_INDEX',
                         geometry_list=[parcel_bbox],
                         time=('2018-05-01', '2018-05-31'),
                         resolution='50m',
                         bins=5,
                         instance_id=INSTANCE_ID)

In [ ]:
fis_parcel_data = fis_parcel_request.get_data()
fis_parcel_data[0]['C0'][0]

In [ ]:
df_parcel = fis_data_to_dataframe(fis_parcel_data)
df_parcel

In [ ]:
wcs_true_color_parcel_request = WcsRequest(layer='NVDI_INDEX',
                                    bbox=parcel_bbox, 
                                    time='2018-05-01', 
                                    resx='50m', resy='50m',
                                    instance_id=INSTANCE_ID)

In [ ]:
wcs_true_color_parcel_data = wcs_true_color_parcel_request.get_data()